<a href="https://colab.research.google.com/github/nakanoelio/I2A2---Bone-Age-Regression/blob/master/Bone_Age_Regression_2Nets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import keras
from keras.models import Model
from keras.layers import *
from keras.optimizers import *
from keras.losses import *
from keras.regularizers import *
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.applications import DenseNet121
from keras.applications import Xception
from keras.callbacks import *

from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import pandas as pd
import matplotlib.image as mpimg
import cv2
import glob
import h5py
import random

import os
from tqdm import tqdm
from PIL import Image
%matplotlib inline

seed = 42

Using TensorFlow backend.


In [ ]:
#os.mkdir('/content/gdrive/My Drive/Kaggle/I2A2_Bone_Regression/NeuralNet_concat_dia2')

In [ ]:
#!unzip -q '/content/gdrive/My Drive/Kaggle/I2A2_Bone_Regression/i2a2-bone-age-regression.zip' -d '/content/gdrive/My Drive/Kaggle/I2A2_Bone_Regression/'

In [ ]:
train_df = pd.read_csv('/content/gdrive/My Drive/Kaggle/I2A2_Bone_Regression/train.csv')
test_df = pd.read_csv('/content/gdrive/My Drive/Kaggle/I2A2_Bone_Regression/test.csv')
print(train_df.shape)
print(test_df.shape)

(12611, 3)
(249, 2)


In [ ]:
#X = []
#for i in tqdm(train_df['fileName']):
    #im = Image.open(os.path.join('../input/i2a2-bone-age-regression/images',i)).convert('RGB')
    #im = im.resize((256,256))
    #im = np.asarray(im)
    #X.append(im)
#X_t = np.stack(X,axis=0) 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_df["fileName"],train_df["boneage"], test_size = 0.20, random_state = seed)

In [ ]:
img_size = (224)

rotation_range = 20
width_shift_range = 0.1
height_shift_range = 0.1
horizontal_flip = True
shear_range = 0.1
zoom_range = 0.1

datagen = ImageDataGenerator(
        rotation_range=rotation_range,
        width_shift_range=width_shift_range,
        height_shift_range=height_shift_range,
        shear_range=shear_range,
        zoom_range=zoom_range,
        horizontal_flip=horizontal_flip,
        fill_mode='nearest',rescale=1./255.,validation_split=0.25)



train_iter=datagen.flow_from_dataframe(dataframe=train_df, directory="/content/gdrive/My Drive/Kaggle/I2A2_Bone_Regression/images", x_col="fileName", y_col="boneage", subset="training", class_mode="raw", target_size=(img_size,img_size), color_mode='rgb')
valid_iter=datagen.flow_from_dataframe(dataframe=train_df, directory="/content/gdrive/My Drive/Kaggle/I2A2_Bone_Regression/images", x_col="fileName", y_col="boneage", subset="validation", class_mode="raw", target_size=(img_size,img_size), color_mode='rgb')


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 1 invalid image filename(s) in x_col="fileName". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 9458 validated image filenames.
Found 3152 validated image filenames.


In [ ]:
#pretrained_model = VGG16(weights="imagenet", include_top=False, input_shape=(img_size,img_size,3))

In [ ]:
#img_size = (224)
#pretrained_model = DenseNet121(include_top=False,weights="imagenet", input_shape=(img_size,img_size,3),)

In [ ]:
#pretrained_model = Xception(include_top=False, weights="imagenet",input_shape=(img_size,img_size,3),)

In [ ]:
#number_of_frozen_layers = 0
#for i, layer in enumerate(pretrained_model.layers):
    #if i>=number_of_frozen_layers:
        #break
    #layer.trainable = False

In [ ]:
img_size = (224)

inputs = keras.Input(shape=(img_size, img_size, 3))
#x = pretrained_model(inputs)
xx = Conv2D(filters=32, kernel_size=3, padding='same', activation='elu')(inputs)
xx = BatchNormalization()(xx)
xx = Conv2D(filters=32, kernel_size=3, padding='same', activation='elu')(xx)
xx = BatchNormalization()(xx)
xx = MaxPooling2D(pool_size=(3, 3), strides=2)(xx)

yy = Conv2D(filters=64, kernel_size=3, padding='same', activation='elu')(xx)
yy = BatchNormalization()(yy)
yy = Conv2D(filters=64, kernel_size=3, padding='same', activation='elu')(yy)
yy = BatchNormalization()(yy)
yy = MaxPooling2D(pool_size=(3, 3), strides=2)(yy)

zz = Conv2D(filters=128, kernel_size=3, padding='same', activation='elu')(yy)
zz = BatchNormalization()(zz)
zz = Conv2D(filters=128, kernel_size=3, padding='same', activation='elu')(zz)
zz = BatchNormalization()(zz)
zz = MaxPooling2D(pool_size=(3, 3), strides=2)(zz)

ll = Conv2D(filters=256, kernel_size=3, padding='same', activation='elu')(zz)
ll = BatchNormalization()(ll)
ll = Conv2D(filters=256, kernel_size=3, padding='same', activation='elu')(ll)
ll = BatchNormalization()(ll)
ll = MaxPooling2D(pool_size=(3, 3), strides=2)(ll)

mm =Conv2D(filters=384, kernel_size=3, padding='same', activation='elu')(ll)
mm = BatchNormalization()(mm)
mm =Conv2D(filters=384, kernel_size=3, padding='same', activation='elu')(mm)
mm = BatchNormalization()(mm)
mm = MaxPooling2D(pool_size=(3, 3), strides=2)(mm)

x = Conv2D(filters=32, kernel_size=3, padding='same', activation='elu')(inputs)
x = BatchNormalization()(x)
x =Conv2D(filters=32, kernel_size=3, padding='same', activation='elu')(x)
x = BatchNormalization()(x)
x = Concatenate(axis=-1)([inputs, x])
x = MaxPooling2D(pool_size=(3, 3), strides=2)(x)

y =Conv2D(filters=64, kernel_size=3, padding='same', activation='elu')(x)
y = BatchNormalization()(y)
y =Conv2D(filters=64, kernel_size=3, padding='same', activation='elu')(y)
y = BatchNormalization()(y)
y = Concatenate(axis=-1)([x, y])
y = MaxPooling2D(pool_size=(3, 3), strides=2)(y)

z =Conv2D(filters=128, kernel_size=3, padding='same', activation='elu')(y)
z = BatchNormalization()(z)
z =Conv2D(filters=128, kernel_size=3, padding='same', activation='elu')(z)
z = BatchNormalization()(z)
z = Concatenate(axis=-1)([y, z])
z = MaxPooling2D(pool_size=(3, 3), strides=2)(z)

l =Conv2D(filters=256, kernel_size=3, padding='same', activation='elu')(z)
l = BatchNormalization()(l)
l =Conv2D(filters=256, kernel_size=3, padding='same', activation='elu')(l)
l = BatchNormalization()(l)
l = Concatenate(axis=-1)([z, l])
l = MaxPooling2D(pool_size=(3, 3), strides=2)(l)

m =Conv2D(filters=384, kernel_size=3, padding='same', activation='elu')(l)
m = BatchNormalization()(m)
m =Conv2D(filters=384, kernel_size=3, padding='same', activation='elu')(m)
m = BatchNormalization()(m)
m = Concatenate(axis=-1)([l,m])
m = MaxPooling2D(pool_size=(3, 3), strides=2)(m)

n = Concatenate(axis=-1)([m,mm])
n = GlobalAveragePooling2D()(n)

#x = Flatten()(x)
f = Dropout(rate=0.2, seed=42)(n)
f = Dense(2048, activation  =  'elu')(f)
f = Dropout(rate=0.2, seed=42)(f)
f = Dense(2048, activation  =  'elu')(f)
f = Dense(1, activation  =  'linear')(f)

model = Model(inputs, f)
model.summary()

#for layer in model.layers:
#  layer.kernel_regularizer = l2(1e-4)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 224, 224, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_11 (BatchNo (None, 224, 224, 32) 128         conv2d_11[0][0]                  
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 224, 224, 32) 9248        batch_normalization_11[0][0]     
____________________________________________________________________________________________

In [ ]:
batch_size = 40
num_epochs = 100
learning_rate = 1e-3

weights_filepath = '/content/gdrive/My Drive/Kaggle/I2A2_Bone_Regression/NeuralNet_concat/model.{epoch:02d}-{val_loss:.2f}.h5'

callbacks = [ModelCheckpoint(weights_filepath, monitor='loss', mode='min', verbose=1, save_best_only=True), ReduceLROnPlateau(monitor="loss", factor=0.1, patience=10, verbose=1, mode="auto", min_delta=0.0001, cooldown=0, min_lr=0,)]
#EarlyStopping(monitor='val_loss', mode='min', patience=10), 
mse = keras.losses.MeanSquaredError()        

model.compile(loss = mse, optimizer = Adam(lr = learning_rate), metrics = ['acc'])

In [ ]:
weights_filepath = model.load_weights('/content/gdrive/My Drive/Kaggle/I2A2_Bone_Regression/NeuralNet_concat/model.44-820.58.h5')

STEP_SIZE_TRAIN=train_iter.n//train_iter.batch_size
STEP_SIZE_VALID=valid_iter.n//valid_iter.batch_size
history = model.fit_generator(generator=train_iter, steps_per_epoch=STEP_SIZE_TRAIN, validation_data=valid_iter, validation_steps=STEP_SIZE_VALID, epochs=num_epochs, verbose = 1, callbacks=callbacks)

Epoch 1/100
 80/295 [=======>......................] - ETA: 2:08:31 - loss: 185.1559 - acc: 0.0271

KeyboardInterrupt: ignored

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.ylabel('acurácia')
plt.xlabel('época')
plt.legend(['treino', 'validação'], loc = 'upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('loss')
plt.xlabel('época')
plt.legend(['treino', 'validação'], loc = 'upper left')
plt.show()

In [ ]:
X = []
for i in tqdm(X_test):
    im = Image.open(os.path.join('/content/gdrive/My Drive/Kaggle/I2A2_Bone_Regression/images',i)).convert('RGB')
    im = im.resize((img_size,img_size))
    im = np.asarray(im)
    X.append(im)
X_tt = np.stack(X,axis=0) 

model.evaluate(X_tt, y_test)



In [ ]:
model.save("/content/gdrive/My Drive/Kaggle/I2A2_Bone_Regression/NeuralNet/my_h5_model.h5")

In [ ]:
img_size = 224
X = []
for i in tqdm(test_df['fileName']):
    im = Image.open(os.path.join('/content/gdrive/My Drive/Kaggle/I2A2_Bone_Regression/preview',i)).convert('RGB')
    im = im.resize((img_size,img_size))
    im = np.asarray(im)/255
    X.append(im)
X_t = np.stack(X,axis=0) 

100%|██████████| 249/249 [00:06<00:00, 41.35it/s]


In [ ]:
model = tf.keras.models.load_model('/content/my_h5_model.h5')
model.load_weights('/content/my_h5_model.h5')
y_t = model.predict(X_t,verbose=1)
print(y_t)

8/8 [==============================] - 1s 63ms/step
[[ 51.848618]
 [ 54.73187 ]
 [ 61.400948]
 [105.24501 ]
 [169.9654  ]
 [ 53.89872 ]
 [ 58.844116]
 [104.86145 ]
 [ 79.05908 ]
 [ 73.95665 ]
 [ 56.938923]
 [ 83.264084]
 [ 82.680695]
 [ 67.86927 ]
 [ 77.08283 ]
 [118.92383 ]
 [ 67.710365]
 [153.71645 ]
 [ 69.88101 ]
 [146.30162 ]
 [ 44.282917]
 [ 60.0177  ]
 [ 53.976593]
 [ 57.14627 ]
 [ 60.298954]
 [170.39519 ]
 [ 84.61151 ]
 [ 36.73858 ]
 [ 56.263977]
 [ 96.00036 ]
 [ 93.79335 ]
 [139.06567 ]
 [ 43.049408]
 [ 77.23334 ]
 [110.32254 ]
 [ 67.15832 ]
 [ 80.61116 ]
 [ 96.95723 ]
 [182.97418 ]
 [ 67.89435 ]
 [136.39629 ]
 [121.001335]
 [ 43.75162 ]
 [ 40.677666]
 [147.6081  ]
 [127.75756 ]
 [127.88212 ]
 [132.42165 ]
 [ 61.65413 ]
 [138.9972  ]
 [ 66.181725]
 [190.11996 ]
 [ 71.23677 ]
 [ 49.576195]
 [ 57.582874]
 [ 65.21706 ]
 [ 42.72416 ]
 [145.7449  ]
 [128.19875 ]
 [ 68.58675 ]
 [ 55.942753]
 [ 44.925175]
 [127.60368 ]
 [ 53.87229 ]
 [ 67.01149 ]
 [ 81.20555 ]
 [111.21135 ]
 [ 66.0722

In [ ]:
test_df['boneage'] = y_t
test_df.drop(['patientSex'],axis=1,inplace=True)
print(test_df.head())
test_df.to_csv('/content/gdrive/My Drive/Kaggle/I2A2_Bone_Regression/submission.csv',index=False)

                                   fileName     boneage
0  8B2563DD-1B74-4F22-A2DC-46CCF7B58920.png   51.848618
1  34E48F6E-144E-4E36-9996-71B797A5E348.png   54.731869
2  601B717A-3548-42E2-BBB7-20C9DB598332.png   61.400948
3  933BBD2E-2E42-4E46-8CE6-EE89688D9C1E.png  105.245010
4  992B3882-02C9-493D-BC4A-C4015E1297A3.png  169.965393
